install packagees

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

prepare/process data
 - normalize numerical col
 - create one hot encoding
 - create feature matric

In [ ]:
data = pd.read_csv("cleaned_data.csv")
print

create vae model design
 - encoder
 - decoder
 - define loss function

Training VAE
 - train model
  - forward pas
  - compute loss
  - backpropogation
  - run for # of epochs

Generate Latent Representations
- evaluate user preferences
- compare with other movies

Content-Based Recomendations
 - Compute Similarities
 - generate reccomendations


Testing and Validations
 - Evaluate Reccomendations
 - Fine Tune Model